In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import pycaret.classification as pc

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool, EShapCalcType, EFeaturesSelectionAlgorithm, cv
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

## Препроцессинг

In [10]:
df = dd.read_parquet('..\\full_data\\figma_plan\\full_gender_data_txt.parquet').compute()

# добавляем target, удаляем nan
df = df.loc[:,df.columns[12:-1].to_list()+['user_id']]
df = df.loc[:,df.columns.isin(summary['selected_features_names']+['user_id'])]
df = df.merge(pd.read_parquet('..\\full_data\\public_train.pqt',columns=['user_id','is_male'])[['is_male','user_id']], on = 'user_id', how = 'inner')
df = df.loc[~(df['is_male'].isna()) & (df['is_male'] != 'NA')]
df['is_male'] = df['is_male'].astype('int8')

display(df.head())
display(df.info())

,googleads.g.doubleclick.net,yandex.ru,i.ytimg.com,vk.com,avatars.mds.yandex.net,ad.mail.ru,yastatic.net,apple.com,instagram.com,ads.adfox.ru,...,manrule.ru,programma-peredach.com,mcpehub.org,go64.ru,nail-styles.ru,other_url,other_female_urls_frac,other_male_urls_frac,user_id,is_male
0,0.196891,0.031088,0.025907,0.041451,0.046632,0.031088,0.051813,0.0,0.000000,0.025907,...,0.0,0.0,0.0,0.0,0.0,0.031088,0.005181,0.000000,0,0
1,0.072588,0.112703,0.040115,0.023878,0.104107,0.034384,0.081184,0.0,0.000000,0.019102,...,0.0,0.0,0.0,0.0,0.0,0.013372,0.001910,0.000955,1,0
2,0.092457,0.046229,0.126521,0.141119,0.072993,0.038929,0.046229,0.0,0.000000,0.026764,...,0.0,0.0,0.0,0.0,0.0,0.002433,0.000000,0.000000,2,0
3,0.021818,0.061818,0.007273,0.178182,0.094545,0.080000,0.069091,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.010909,0.003636,0.003636,3,1
4,0.061776,0.030888,0.048906,0.083655,0.046332,0.061776,0.045045,0.0,0.003861,0.028314,...,0.0,0.0,0.0,0.0,0.0,0.011583,0.003861,0.000000,4,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 264326 entries, 0 to 269999
Columns: 402 entries, googleads.g.doubleclick.net to is_male
dtypes: float32(400), int32(1), int8(1)
memory usage: 406.6 MB


None

In [ ]:
display(df.isna().sum().sum())
for name in df['cpe_manufacturer_name'].unique():
    df.loc[(df['cpe_manufacturer_name']==name)&(df['price'].isna()),'price'] = \
        df.loc[(df['cpe_manufacturer_name']==name)&(~df['price'].isna()),'price'].median()
df.loc[df['price']<1000,'price'] = 1000
display(df.isna().sum().sum())

In [17]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['is_male','user_id'], axis = 1),df['is_male'],
                                                    test_size = 0.33,stratify=df['is_male'])#random_state = 42

In [20]:
pool_train = Pool(x_train, y_train,
                  cat_features = list(x_train.select_dtypes(include=['object']).columns))
pool_test = Pool(x_test,y_test,
                 cat_features = list(x_train.select_dtypes(include=['object']).columns))

model = CatBoostClassifier(iterations=1000,
                           random_state=42,
                           task_type='GPU')

model.fit(pool_train, verbose=100,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.025358
0:	learn: 0.6861095	total: 12.1ms	remaining: 12.1s
100:	learn: 0.5295252	total: 1.47s	remaining: 13.1s
200:	learn: 0.5046246	total: 2.67s	remaining: 10.6s
300:	learn: 0.4917290	total: 4.19s	remaining: 9.73s
400:	learn: 0.4830244	total: 5.17s	remaining: 7.72s
500:	learn: 0.4764906	total: 6.11s	remaining: 6.09s
600:	learn: 0.4709506	total: 7.05s	remaining: 4.68s
700:	learn: 0.4663392	total: 8s	remaining: 3.41s
800:	learn: 0.4623531	total: 8.92s	remaining: 2.22s
900:	learn: 0.4588211	total: 9.83s	remaining: 1.08s
999:	learn: 0.4555286	total: 10.8s	remaining: 0us


In [5]:
pool_train = Pool(x_train, y_train,
                  cat_features = list(x_train.select_dtypes(include=['object']).columns))
pool_test = Pool(x_test,y_test,
                 cat_features = list(x_train.select_dtypes(include=['object']).columns))

model = CatBoostClassifier(iterations=1000,
                           random_state=42,
                           task_type='GPU')
summary = model.select_features(
    pool_train,
    eval_set=pool_test,
    features_for_select=list(range(pool_train.num_col())),
    num_features_to_select=400,
    steps=3,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
    shap_calc_type=EShapCalcType.Regular,
    train_final_model=True,
    plot=True,
    verbose=False
)
print('Selected features:', summary['selected_features_names'])

#model.fit(pool_train, eval_set=(x_test,y_test), verbose=100,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.045859
Step #1 out of 3
bestTest = 0.4544962716
bestIteration = 999
Feature #1407 eliminated
Feature #1343 eliminated
Feature #1993 eliminated
Feature #1692 eliminated
Feature #2524 eliminated
Feature #1511 eliminated
Feature #1916 eliminated
Feature #1506 eliminated
Feature #725 eliminated
Feature #2582 eliminated
Feature #350 eliminated
Feature #2486 eliminated
Feature #1687 eliminated
Feature #937 eliminated
Feature #1589 eliminated
Feature #2973 eliminated
Feature #2253 eliminated
Feature #560 eliminated
Feature #2671 eliminated
Feature #1340 eliminated
Feature #1110 eliminated
Feature #1020 eliminated
Feature #2186 eliminated
Feature #2075 eliminated
Feature #2526 eliminated
Feature #2380 eliminated
Feature #2807 eliminated
Feature #217 eliminated
Feature #884 eliminated
Feature #2051 eliminated
Feature #2761 eliminated
Feature #14 eliminated
Feature #49 eliminated
Feature #60 eliminated
Feature #61 eliminated
Feature #69 eliminated
Feature #294 eliminated
F

Selected features: ['googleads.g.doubleclick.net', 'yandex.ru', 'i.ytimg.com', 'vk.com', 'avatars.mds.yandex.net', 'ad.mail.ru', 'yastatic.net', 'apple.com', 'instagram.com', 'ads.adfox.ru', 'google.com', 'mail.yandex.ru', 'tpc.googlesyndication.com', 'zen.yandex.ru', 'online.sberbank.ru', 'ad.adriver.ru', 's0.2mdn.net', 'sun9-32.userapi.com', 'sun9-17.userapi.com', 'sun9-29.userapi.com', 'ads.betweendigital.com', 'sun9-27.userapi.com', 'sun9-61.userapi.com', 'play.google.com', 'm.vk.com', 'o2.mail.ru', 'sun9-80.userapi.com', 'sun9-88.userapi.com', 'imasdk.googleapis.com', 'ok.ru', 'node3.online.sberbank.ru', 'connect.ok.ru', 'id.tinkoff.ru', 's3.amazonaws.com', 'm.avito.ru', 'cstatic.weborama.fr', 'esia.gosuslugi.ru', 'st.mycdn.me', 'm.ok.ru', 'moevideo.biz', 'smi2.ru', 'banners.adfox.ru', 'i.pinimg.com', 'frontend.vh.yandex.ru', 'playreplay.me', 'pub-eu.p.otm-r.com', 'yhb.p.otm-r.com', 'ssp.rambler.ru', 't.me', 'go.mail.ru', 'my.pochtabank.ru', 'online.vtb.ru', 'login.mts.ru', 'duckd

In [7]:
np.save('top400_features.npy', summary['selected_features_names'], allow_pickle=True)

In [21]:
print(f'GINI по полу {2 * roc_auc_score(y_test, model.predict_proba(x_test)[:,1]) - 1:2.6f}')
print(classification_report(y_test, model.predict(x_test)))

GINI по полу 0.727156
              precision    recall  f1-score   support

           0       0.77      0.77      0.77     42568
           1       0.78      0.78      0.78     44660

    accuracy                           0.78     87228
   macro avg       0.78      0.78      0.78     87228
weighted avg       0.78      0.78      0.78     87228



In [5]:
x = df.drop(['is_male','user_id'],axis=1)
y = df['is_male']

params = {'iterations': 1000,
          'random_state': 42,
          'task_type': 'GPU',
          'loss_function': 'Logloss',
          'custom_loss':'AUC',
          'eval_loss':'AUC',
          'learning_rate': 0.05}

cv_fit = cv(
    params = params,
    pool = Pool(x,label = y,cat_features = list(x.select_dtypes(include=['object']).columns)),
    fold_count = 5,
    shuffle = True,
    partition_random_seed = 42,
    stratified = True,
    verbose = False,
    plot = True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/options/plain_options_helper.cpp:511: Unknown option {eval_loss} with value "AUC"